About the dataset

1) id => unique id for a news article
2) title => the title of a news article
3) author => author of the news articel
4) text => news article
5) label => a label that marks whether the news article is real or fake 

In [62]:
""" Imports """
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [63]:
# download stopwords from nltk
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sohail\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [64]:
# get english stop words
english_sw=stopwords.words('english')
english_sw

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [65]:
""" load the data """
news_data=pd.read_csv('train.csv')

In [66]:
""" first five rows of data """
news_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [67]:
""" shape of data """
news_data.shape

(20800, 5)

In [68]:
""" null count """
news_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [69]:
""" fill the null data with empty string ('')  """
news_data=news_data.fillna('')
news_data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [70]:
""" merge the author name with article title seperated by a ',' """
news_data['content']=news_data['author']+', '+news_data['title']
news_data.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,"Darrell Lucus, House Dem Aide: We Didn’t Even ..."
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn, FLYNN: Hillary Clinton, Big W..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,"Consortiumnews.com, Why the Truth Might Get Yo..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,"Jessica Purkiss, 15 Civilians Killed In Single..."
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,"Howard Portnoy, Iranian woman jailed for ficti..."


In [71]:
""" seperate the labels and values """
X=news_data.drop(columns='label', axis=1)
Y=news_data['label']

In [72]:
print(X.head())
print(Y.head())

   id                                              title              author  \
0   0  House Dem Aide: We Didn’t Even See Comey’s Let...       Darrell Lucus   
1   1  FLYNN: Hillary Clinton, Big Woman on Campus - ...     Daniel J. Flynn   
2   2                  Why the Truth Might Get You Fired  Consortiumnews.com   
3   3  15 Civilians Killed In Single US Airstrike Hav...     Jessica Purkiss   
4   4  Iranian woman jailed for fictional unpublished...      Howard Portnoy   

                                                text  \
0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1  Ever get the feeling your life circles the rou...   
2  Why the Truth Might Get You Fired October 29, ...   
3  Videos 15 Civilians Killed In Single US Airstr...   
4  Print \nAn Iranian woman has been sentenced to...   

                                             content  
0  Darrell Lucus, House Dem Aide: We Didn’t Even ...  
1  Daniel J. Flynn, FLYNN: Hillary Clinton, Big W...  
2  Consortiumnews

In [73]:
port_stem=PorterStemmer()

In [74]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content) # accept only a-z charaters
  stemmed_content=stemmed_content.lower() # convert them to lowercase
  stemmed_content=stemmed_content.split() # convert to list
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in english_sw]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [75]:
news_data['content'] = news_data['content'].apply(stemming)
news_data['content'].head() # removed all filler words like i, me, you etc and converted to lowercase

0    darrel lucu hous dem aid even see comey letter...
1    daniel j flynn flynn hillari clinton big woman...
2               consortiumnew com truth might get fire
3    jessica purkiss civilian kill singl us airstri...
4    howard portnoy iranian woman jail fiction unpu...
Name: content, dtype: object

In [76]:
""" again seperating the X and Y """
X=news_data['content'].values
Y=news_data['label'].values
# assigned X and Y with list 
print(X)
print(Y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


In [77]:
print(X.shape)
print(Y.shape)

(20800,)
(20800,)


In [78]:
# ML model only understand numeric data not text data
vectorizer= TfidfVectorizer()
vectorizer.fit(X)
X=vectorizer.transform(X)

In [79]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

Splitting the X and Y data into testing and training data

In [80]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

Model Training using Logistic Regression

In [81]:
model = LogisticRegression()

In [82]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

In [83]:
""" accuracy score for training data """
x_train_predictions=model.predict(X_train)
x_train_accuracy=accuracy_score(x_train_predictions,Y_train)
x_train_accuracy

0.9865985576923076

In [84]:
""" accuracy score for testing data """
x_test_predictions=model.predict(X_test)
x_test_accuracy=accuracy_score(x_test_predictions,Y_test)
x_test_accuracy

0.9790865384615385

0 => Real News
1 => Fake News

In [85]:
# 3 => real
# 5 => fake

index=3

X_new = X_test[index]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

print(Y_test[index])

[0]
The news is Real
0
